# TA-Lib Tutorial

"TA-Lib is widely used by trading software developers requiring to perform technical analysis of financial market data."  
We will show how to use them here, in the context of pinkfish.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import talib
from talib.abstract import *
from talib import MA_Type

import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.3f}'.format

%matplotlib inline

In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [4]:
symbol = 'SPY'
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from cache, if available.

In [5]:
ts = pf.fetch_timeseries(symbol)
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2021-03-22,394.070,389.970,390.030,392.590,73778600.000,392.590
2021-03-23,393.460,388.660,391.910,389.500,90686600.000,389.500
2021-03-24,392.750,387.470,391.000,387.520,97588600.000,387.520
2021-03-25,390.550,383.900,385.980,389.700,116128600.000,389.700
2021-03-26,396.410,390.290,390.930,395.980,113023400.000,395.980


Select timeseries between start and end.

In [6]:
ts = pf.select_tradeperiod(ts, start, end)
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
2017-01-03,225.830,223.880,225.040,225.240,91366500.000,208.214
2017-01-04,226.750,225.610,225.620,226.580,78744400.000,209.452
2017-01-05,226.580,225.480,226.270,226.400,78379000.000,209.286
2017-01-06,227.750,225.900,226.530,227.210,71559900.000,210.035
2017-01-09,227.070,226.420,226.910,226.460,46939700.000,209.341


### Get info about TA-Lib

In [7]:
print('There are {} TA-Lib functions!'.format(len(talib.get_functions())))

There are 158 TA-Lib functions!


Here is a complete listing of the functions by group:

In [8]:
for group, funcs in talib.get_function_groups().items():
    print(group)
    print('-----------------------------------------')
    for func in funcs:
        f = Function(func)
        print('{} - {}'.format(func, f.info['display_name']))
    print()

Cycle Indicators
-----------------------------------------
HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
HT_PHASOR - Hilbert Transform - Phasor Components
HT_SINE - Hilbert Transform - SineWave
HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode

Math Operators
-----------------------------------------
ADD - Vector Arithmetic Add
DIV - Vector Arithmetic Div
MAX - Highest value over a specified period
MAXINDEX - Index of highest value over a specified period
MIN - Lowest value over a specified period
MININDEX - Index of lowest value over a specified period
MINMAX - Lowest and highest values over a specified period
MINMAXINDEX - Indexes of lowest and highest values over a specified period
MULT - Vector Arithmetic Mult
SUB - Vector Arithmetic Substraction
SUM - Summation

Math Transform
-----------------------------------------
ACOS - Vector Trigonometric ACos
ASIN - Vector Trigonometric ASin
ATAN - Vector Trigonometric AT

### Get info about a specific TA-Lib function

There are 2 different API that are available with talib, namely Function API and Abstract API.  For the Function API, you pass in a price series.  For the Abstract API, you pass in a collection of named inputs: 'open', 'high', 'low', 'close', and 'volume'.  One or more of these may be used as defaults, but can be changed with the 'price' parameter.  

Print the function instance to get documentation.  We see that SMA has the parameter 'timeperiod' with default '30'.  The input_arrays can be a dataframe with columns named 'open', 'high', 'low', 'close', and 'volume'.

In [9]:
print(SMA)

SMA([input_arrays], [timeperiod=30])

Simple Moving Average (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 30
Outputs:
    real


More information is available through the 'info' property.  We observe here that the default price used is 'close'.  This can be changed by setting 'price' in the function call, e.g. price='open'.

In [10]:
print(SMA.info)

{'name': 'SMA', 'group': 'Overlap Studies', 'display_name': 'Simple Moving Average', 'function_flags': ['Output scale same as input'], 'input_names': OrderedDict([('price', 'close')]), 'parameters': OrderedDict([('timeperiod', 30)]), 'output_flags': OrderedDict([('real', ['Line'])]), 'output_names': ['real']}


If we just want to see the inputs, we can print the input_names property.

In [11]:
print(SMA.input_names)

OrderedDict([('price', 'close')])


If we just want to see the parameters, we can print the paramters property.

In [12]:
print(SMA.parameters)

OrderedDict([('timeperiod', 30)])


If we just want to see the outputs, we can print the output_names property.

In [13]:
print(SMA.output_names)

['real']


### Create a technical indicator using talib

Create technical indicator: SMA (using defaults: timeperiod=30, price='close')

In [14]:
sma = SMA(ts)
sma.tail()

date
2021-03-22   389.189
2021-03-23   389.156
2021-03-24   389.065
2021-03-25   389.052
2021-03-26   389.228
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='close')

In [15]:
sma200 = SMA(ts, timeperiod=200)
sma200.tail()

date
2021-03-22   351.383
2021-03-23   351.734
2021-03-24   352.055
2021-03-25   352.400
2021-03-26   352.785
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='open')

In [16]:
sma200 = SMA(ts, timeperiod=200, price='open')
sma200.tail()

date
2021-03-22   351.472
2021-03-23   351.845
2021-03-24   352.199
2021-03-25   352.527
2021-03-26   352.875
dtype: float64

### Add a technical indicator to a pinkfish timeseries

In [17]:
ts['sma200'] = sma200
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2021-03-22,394.070,389.970,390.030,392.590,73778600.000,392.590,351.472
2021-03-23,393.460,388.660,391.910,389.500,90686600.000,389.500,351.845
2021-03-24,392.750,387.470,391.000,387.520,97588600.000,387.520,352.199
2021-03-25,390.550,383.900,385.980,389.700,116128600.000,389.700,352.527
2021-03-26,396.410,390.290,390.930,395.980,113023400.000,395.980,352.875


### Try another one

Commodity Channel Index

In [18]:
print(CCI)

CCI([input_arrays], [timeperiod=14])

Commodity Channel Index (Momentum Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    timeperiod: 14
Outputs:
    real


In [19]:
print(CCI.input_names)

OrderedDict([('prices', ['high', 'low', 'close'])])


In [20]:
print(CCI.parameters)

OrderedDict([('timeperiod', 14)])


In [21]:
cci = CCI(ts)
ts['cci'] = cci
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci
date,,,,,,,,
2021-03-22,394.070,389.970,390.030,392.590,73778600.000,392.590,351.472,38.716
2021-03-23,393.460,388.660,391.910,389.500,90686600.000,389.500,351.845,10.094
2021-03-24,392.750,387.470,391.000,387.520,97588600.000,387.520,352.199,-27.628
2021-03-25,390.550,383.900,385.980,389.700,116128600.000,389.700,352.527,-72.042
2021-03-26,396.410,390.290,390.930,395.980,113023400.000,395.980,352.875,58.510


### Now for something a little more difficult

Bollinger Bands

In [22]:
print(BBANDS)

BBANDS([input_arrays], [timeperiod=5], [nbdevup=2], [nbdevdn=2], [matype=0])

Bollinger Bands (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 5
    nbdevup: 2
    nbdevdn: 2
    matype: 0 (Simple Moving Average)
Outputs:
    upperband
    middleband
    lowerband


In [23]:
print(BBANDS.input_names)

OrderedDict([('price', 'close')])


In [24]:
print(BBANDS.parameters)

OrderedDict([('timeperiod', 5), ('nbdevup', 2), ('nbdevdn', 2), ('matype', 0)])


Print the available moving average types

In [25]:
attributes = [attr for attr in dir(MA_Type) 
              if not attr.startswith('__')]
attributes

['DEMA', 'EMA', 'KAMA', 'MAMA', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', '_lookup']

In [26]:
MA_Type.__dict__

{'_lookup': {0: 'Simple Moving Average',
  1: 'Exponential Moving Average',
  2: 'Weighted Moving Average',
  3: 'Double Exponential Moving Average',
  4: 'Triple Exponential Moving Average',
  5: 'Triangular Moving Average',
  6: 'Kaufman Adaptive Moving Average',
  7: 'MESA Adaptive Moving Average',
  8: 'Triple Generalized Double Exponential Moving Average'}}

In [27]:
print(MA_Type[MA_Type.DEMA])

Double Exponential Moving Average


Set timeperiod=20 and matype=MA_Type.EMA

In [28]:
#upper, middle, lower = BBANDS(ts, timeperiod=20, matype=MA_Type.EMA)
#(for some reason, the abstract API doesn't work for BBANDS, so use the function API)

upper, middle, lower = talib.BBANDS(ts.close, timeperiod=20, matype=MA_Type.EMA)
ts['upper'] = upper; ts['middle'] = middle; ts['lower'] = lower
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower
date,,,,,,,,,,,
2021-03-22,394.070,389.970,390.030,392.590,73778600.000,392.590,351.472,38.716,401.324,389.835,378.347
2021-03-23,393.460,388.660,391.910,389.500,90686600.000,389.500,351.845,10.094,401.292,389.803,378.315
2021-03-24,392.750,387.470,391.000,387.520,97588600.000,387.520,352.199,-27.628,400.986,389.586,378.186
2021-03-25,390.550,383.900,385.980,389.700,116128600.000,389.700,352.527,-72.042,400.667,389.597,378.526
2021-03-26,396.410,390.290,390.930,395.980,113023400.000,395.980,352.875,58.510,401.010,390.205,379.399


In [29]:
print(MOM)

MOM([input_arrays], [timeperiod=10])

Momentum (Momentum Indicators)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 10
Outputs:
    real


In [30]:
mom10 = MOM(ts, timeperiod=10)
mom10.tail()

date
2021-03-22   10.870
2021-03-23    2.330
2021-03-24   -2.060
2021-03-25   -3.830
2021-03-26    1.920
dtype: float64

In [31]:
ts['mom10'] = mom10
ts.head(50)

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower,mom10
date,,,,,,,,,,,,
2017-01-03,225.830,223.880,225.040,225.240,91366500.000,208.214,nan,nan,nan,nan,nan,nan
2017-01-04,226.750,225.610,225.620,226.580,78744400.000,209.452,nan,nan,nan,nan,nan,nan
2017-01-05,226.580,225.480,226.270,226.400,78379000.000,209.286,nan,nan,nan,nan,nan,nan
2017-01-06,227.750,225.900,226.530,227.210,71559900.000,210.035,nan,nan,nan,nan,nan,nan
2017-01-09,227.070,226.420,226.910,226.460,46939700.000,209.341,nan,nan,nan,nan,nan,nan
2017-01-10,227.450,226.010,226.480,226.460,63771900.000,209.341,nan,nan,nan,nan,nan,nan
2017-01-11,227.100,225.590,226.360,227.100,74650000.000,209.933,nan,nan,nan,nan,nan,nan
2017-01-12,226.750,224.960,226.500,226.530,72113200.000,209.406,nan,nan,nan,nan,nan,nan
2017-01-13,227.400,226.690,226.730,227.050,62717900.000,209.887,nan,nan,nan,nan,nan,nan


In [32]:
m1 = ts['close'].pct_change(periods=10)
ts['m1'] = m1
ts.head(50)

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower,mom10,m1
date,,,,,,,,,,,,,
2017-01-03,225.830,223.880,225.040,225.240,91366500.000,208.214,nan,nan,nan,nan,nan,nan,nan
2017-01-04,226.750,225.610,225.620,226.580,78744400.000,209.452,nan,nan,nan,nan,nan,nan,nan
2017-01-05,226.580,225.480,226.270,226.400,78379000.000,209.286,nan,nan,nan,nan,nan,nan,nan
2017-01-06,227.750,225.900,226.530,227.210,71559900.000,210.035,nan,nan,nan,nan,nan,nan,nan
2017-01-09,227.070,226.420,226.910,226.460,46939700.000,209.341,nan,nan,nan,nan,nan,nan,nan
2017-01-10,227.450,226.010,226.480,226.460,63771900.000,209.341,nan,nan,nan,nan,nan,nan,nan
2017-01-11,227.100,225.590,226.360,227.100,74650000.000,209.933,nan,nan,nan,nan,nan,nan,nan
2017-01-12,226.750,224.960,226.500,226.530,72113200.000,209.406,nan,nan,nan,nan,nan,nan,nan
2017-01-13,227.400,226.690,226.730,227.050,62717900.000,209.887,nan,nan,nan,nan,nan,nan,nan
